# 15.095 Homework 2  -  October 10, 2018
### Kim-Anh-Nhi Nguyen  -  MIT ID: 913785552

## 1. Algorithmic Framework for Linear Regression

### a. Read the data sets and separate into training, validation and testing sets (50%, 25% and 25% respectively)

In [1]:
using JuMP, Gurobi, DataFrames

In [2]:
df1 = readtable("data1.csv", header=true);
df2 = readtable("data2.csv", header=true);

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] #readtable#232(::Bool, ::Char, ::Array{Char,1}, ::Char, ::Array{String,1}, ::Array{String,1}, ::Array{String,1}, ::Bool, ::Int64, ::Array{Symbol,1}, ::Array{Any,1}, ::Bool, ::Char, ::Bool, ::Int64, ::Array{Int64,1}, ::Bool, ::Symbol, ::Bool, ::Bool, ::DataFrames.#readtable, ::String) at C:\Users\utilisateur\.julia\v0.6\DataFrames\src\deprecated.jl:1045
 [3] (::DataFrames.#kw##readtable)(::Array{Any,1}, ::DataFrames.#readtable, ::String) at .\<missing>:0
 [4] include_string(::String, ::String) at .\loading.jl:522
 [5] include_string(::Module, ::String, ::String) at C:\Users\utilisateur\.julia\v0.6\Compat\src\Compat.jl:88
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\utilisateur\.julia\v0.6\IJulia\src\execute_request.jl:180
 [7] (::Compat.#inner#14{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at C:\Users\utilisateur\.julia\v0.6\Compat\src\Compat.jl:332
 [8] eventloop(::ZMQ.Sock

In [3]:
function separate_dataset(dataset)
    show=true

    X, y = Matrix(dataset[:,1:end-1]), dataset[:,end]
    
    # Split into training, validation and test sets (50%/25%/25%)
    n,p = size(X)
    val_start, test_start = round(Int, 0.50 * n), round(Int, 0.75 * n)
    
    X_train, y_train = X[1:val_start - 1, :], y[1:val_start - 1]
    X_val, y_val = X[val_start:test_start - 1, :], y[val_start:test_start - 1]
    X_test, y_test= X[test_start:end, :], y[test_start:end]
    
    if show==true
        #See the size of training, validation and testing sets
        println("\nSize of training set:",size(X_train),size(y_train))
        println("Size of validation set:",size(X_val),size(y_val))
        println("Size of test set:",size(X_test),size(y_test))
    end
    
    return(X_train,y_train, X_val, y_val, X_test, y_test)
end;

### Apply nonlinear transformations

In [23]:
transforms = [x -> x, x -> x.^2, x -> sqrt.(x), x -> log.(x)]
function transformations(X_train, X_val, X_test)
    show=true
    p_orig = size(X_train,2)
    apply_transforms(X) = hcat([transforms[i](X) for i = 1:length(transforms)]...)
    p2 = p_orig * length(transforms)
    X_train2 = apply_transforms(X_train)
    X_val2 = apply_transforms(X_val)
    X_test2 = apply_transforms(X_test)
    
    if show==true
        #See the size of training, validation and testing sets
        println("\nNew size of training set:",size(X_train2))
        println("New size of validation set:",size(X_val2))
        println("New size of test set:",size(X_test2))
    end
    
    return(p_orig, X_train2, X_val2, X_test2)
end;   

In [24]:
X_train1,y_train1, X_val1, y_val1, X_test1, y_test1 = separate_dataset(df1)
p_orig1, X_train1, X_val1, X_test1 = transformations(X_train1, X_val1, X_test1)

X_train2,y_train2, X_val2, y_val2, X_test2, y_test2 = separate_dataset(df2)
p_orig2, X_train2, X_val2, X_test2 = transformations(X_train2, X_val2, X_test2);


Size of training set:(124, 15)(124,)
Size of validation set:(63, 15)(63,)
Size of test set:(63, 15)(63,)

New size of training set:(124, 60)
New size of validation set:(63, 60)
New size of test set:(63, 60)

Size of training set:(249, 7)(249,)
Size of validation set:(125, 7)(125,)
Size of test set:(126, 7)(126,)

New size of training set:(249, 28)
New size of validation set:(125, 28)
New size of test set:(126, 28)


### Compute the correlations

In [7]:
function sorted_correlations(cor_matrix)
    c = copy(abs.(cor_matrix))
    p = length(cor_matrix[1,:])
    num_pairs = convert(Int64,p*(p-1)/2)
    pair_list = zeros(Int64, num_pairs, 2)
    magnitude = zeros(num_pairs)

    # Set lower triangular correlation values = 0
    for i=1:p
        for j=1:i
            c[i,j] = 0
        end
    end

    for i=1:num_pairs
        ind = indmax(c)
        col = floor(ind/p) + 1
        row = ind % p
        magnitude[i] = c[ind]
        pair_list[i,1] = row
        pair_list[i,2] = col
        c[ind] = 0
    end

    return(pair_list, magnitude)
end;

In [8]:
function cor_matrix(p_orig, X_train)
    # Find the correlation matrix of the independent variables of the training data
    cor_matrix = cor(X_train[:,1:p_orig])
    pair_list, magnitude = sorted_correlations(cor_matrix)
    num_pairs = length(magnitude)
    return (pair_list, magnitude, num_pairs)
end;

In [9]:
pair_list1, magnitude1, num_pairs1 = cor_matrix(p_orig1, X_train1)
pair_list2, magnitude2, num_pairs2 = cor_matrix(p_orig2, X_train2);

### Build the MIO model

In [10]:
function linear_regression_framework(p_orig,
                                    X_train,
                                    y_train,
                                    pair_list, magnitude, num_pairs,
                                    K, # sparsity parameter
                                    Γ ,# robustness parameter (type "\Gamma" then hit tab to get the Γ symbol)
                                    ρ # pairwise multicollinearity threshold (type "\rho" then hit tab to get the ρ symbol)
                                    )
    n, p = size(X_train)
    Big_M = fill(1000, p)
    group_sparsity = true
    groups = ([1 2 3 4], [5 6 7], [8 9 10 11])
    num_opt = length(transforms)
    print_model = false
    OutputFlag = 0
    robustness = true

    m = Model(solver = GurobiSolver(OutputFlag=OutputFlag))

    @variable(m, β[1:p]); # (type "\beta" then hit tab to get the β symbol)
    @variable(m, z[1:p], Bin);

    # Big M constraints
    @constraint(m, m_gt[d=1:p], β[d] <= Big_M[d] * z[d]);
    @constraint(m, m_lt[d=1:p], -Big_M[d] * z[d] <= β[d]);



    # Pairwise multicolinearity constraint
    for i=1:num_pairs
      if magnitude[i] > ρ
        ind1 = pair_list[i,1]
        ind2 = pair_list[i,2]
        @constraint(m, z[ind1] + z[ind2] + z[ind1 + p_orig] + z[ind2 + p_orig]
                + z[ind1 + 2p_orig] + z[ind2 + 2p_orig] + z[ind1 + 3p_orig] + z[ind2 + 3p_orig] <= 1)
      else
        break
      end
    end

    # Group sparsity constraint
    if group_sparsity
      @constraint(m, g_sparsity[i=1:length(groups),j=2:length(groups[i])], z[groups[i][1]] == z[groups[i][j]])
    end

    # Single choice of nonlinear transformation constraint
    if num_opt > 1
      @constraint(m, non_linear[j=1:p_orig], sum(z[j + t*p_orig] for t = 0:(num_opt-1)) <= 1)
    end

    # Sparsity constraint
    @constraint(m, sparsity, sum(z[d] for d = 1:p) <= K)


    # Objective function
    a = 0
    for i = 1:n
      a += 0.5(y_train[i] - dot(β, vec(X_train[i,:])))^2
    end
    # @variable(m, t >= 0)
    @variable(m, θ >= 0)
    # @constraint(m, norm(y_train - X_train * β) <= t)
    @objective(m, Min, a + Γ * θ)

    # Robustness
    if robustness
      @variable(m, β_0[1:p] >= 0)
      @constraint(m, beta_pos[j=1:p], β_0[j] >= β[j])
      @constraint(m, beta_neg[j=1:p], β_0[j] >= -β[j])
      @constraint(m, sum(β_0) <= θ)
    end

    if print_model
      print(m)
    end
    
    return(m, z, β)
    
end;

In [11]:
function get_K_values(β_soln, K)
    β_soln_indices = convert(Array{Int64,1}, zeros(K)) 
    β_soln_max = zeros(K)
    temp = copy(abs.(β_soln))
    for i in 1:K
        max = findmax(temp)
        β_soln_indices[i]= max[2]
        if β_soln[max[2]]>= 0
            β_soln_max[i] = max[1]
        else
            β_soln_max[i] = -max[1]
        end
        
        temp[β_soln_indices[i]] = 0
    end
    β_soln_real = zeros(length(β_soln))
    for i in 1:K
        β_soln_real[β_soln_indices[i]] = β_soln_max[i]
    end

    return(β_soln_real)
end;

In [12]:
function solve_regression(m, z, β, K, print_result)
    status = solve(m)
    z_soln1 = getvalue(z)
    β_soln1 = getvalue(β)
    z_soln = get_K_values(z_soln1, K)
    β_soln = get_K_values(β_soln1, K)
    if print_result
        println("Status = ", status)
        println("Selected variables = ", find(z_soln))
        println("Regression Equation:")
        println("y = ", join(["$(round(β_soln[i],3)) x[$i]" for i in find(z_soln)], " + "))
    end
    
    return (β_soln, z_soln)
end;

In [13]:
function find_best_hyperparameters(p_orig, X_train, X_val, y_train, y_val, 
                                    pair_list, magnitude, num_pairs,
                                    k_interval, k_step)
    p = size(X_train, 2)
    ρ = 0.9
    
    Γ_list = [0.001, 0.01, 0.1, 1, 2]
    k_list = [k_step*i for i in (k_interval[1]/k_step):(k_interval[2]/k_step)]
    k_list = convert(Array{Int64,1}, k_list)
    println("k_list: ", k_list)
    
    a = length(Γ_list)*length(k_list)
    #instantiate arrays
    scores = zeros(a)
    
    for i in 1:length(Γ_list)
        for j in 1:length(k_list)
            # training on train sets for both regression methods
            m, z1, β1 = linear_regression_framework(p_orig, X_train, y_train, pair_list, magnitude, num_pairs,
                                                    k_list[j], Γ_list[i], ρ);
            β, z = solve_regression(m, z1, β1, k_list[j], false)
            
            # performance metrics on validation sets for both regression methods
            scores[length(k_list)*(i-1) + j] = norm(y_val - X_val * β)
        end        
    end
    Argmin = indmin(scores)
    println(scores)
    Γ_argmin, k_argmin = Argmin÷length(k_list)+1, Argmin%length(k_list)
    if k_argmin == 0
        k_argmin=length(k_list)
    end
    
    return Γ_list[Γ_argmin], k_list[k_argmin]
end;

In [14]:
best_Γ1, best_K_1 = find_best_hyperparameters(p_orig1, X_train1, X_val1, y_train1, y_val1, 
                                                pair_list1, magnitude1, num_pairs1, [1,10], 1);

k_list: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - f

In [15]:
best_Γ2, best_K_2 = find_best_hyperparameters(p_orig2, X_train2, X_val2, y_train2, y_val2, 
                                                pair_list2, magnitude2, num_pairs2, [1,28], 1);

k_list: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use 

In [16]:
println("\nBest Γ for dataset1: ", best_Γ1, "\nBest K for dataset1: ", best_K_1)
println("\nBest Γ for dataset2: ", best_Γ2, "\nBest K for dataset2: ", best_K_2);


Best Γ for dataset1: 0.001
Best K for dataset1: 3

Best Γ for dataset2: 0.01
Best K for dataset2: 7


In [17]:
# retrain the whole model with training and validation sets together
function retrain_linear(p_orig, X_train, y_train, X_val, y_val, pair_list, magnitude, num_pairs, K, Γ, ρ)
    X_train_new = vcat(X_train, X_val)
    y_train_new = vcat(y_train, y_val)
    
    m, z1, β1 = linear_regression_framework(p_orig, X_train_new, y_train_new, pair_list, magnitude, num_pairs,
                                                    K, Γ, ρ);
    β, z = solve_regression(m, z1, β1, K, true)
    return(β, z)
end;

In [18]:
β_opt_1, z_opt_1 = retrain_linear(p_orig1, X_train1, y_train1, X_val1, y_val1,  pair_list1, magnitude1, num_pairs1, best_K_1, best_Γ1, 0.9);

Academic license - for non-commercial use only
Status = Optimal
Selected variables = [19, 27, 56]
Regression Equation:
y = 0.039 x[19] + 0.027 x[27] + 2.645 x[56]


In [19]:
β_opt_2, z_opt_2 = retrain_linear(p_orig2, X_train2, y_train2, X_val2, y_val2, pair_list1, magnitude1, num_pairs1, best_K_2, best_Γ2, 0.9);

Academic license - for non-commercial use only
Status = Optimal
Selected variables = [16, 18, 19, 22, 24, 27, 28]
Regression Equation:
y = 1.857 x[16] + -0.445 x[18] + -0.473 x[19] + 3.433 x[22] + 0.132 x[24] + -0.266 x[27] + 0.432 x[28]


In [20]:
function out_of_sample(X_train, y_train, X_val, y_val, X_test, y_test, β_soln, K, Γ, ρ)    
    # Out of sample testing
    y_hat_train = X_train*β_soln
    R2_train = 1 - sum((y_hat_train - y_train).^2) / sum((mean(y_train) - y_train).^2)
    
    y_hat_val = X_val*β_soln
    R2_val = 1 - sum((y_hat_val - y_val).^2) / sum((mean(y_train) - y_val).^2)
    
    y_hat_test = X_test*β_soln
    R2_test = 1 - sum((y_hat_test - y_test).^2) / sum((mean(y_train) - y_test).^2)
    
    z_soln_manual = abs.(β_soln) .> 0.0000001



    println("***PARAMETERS***")
    println("Sparsity parameter K = ", K)
    println("Pairwise correlation threshold ρ = ", ρ)
    println("Robustness parameter Γ = ", Γ)
    println()
    println("***RESULTS***")
    println("MIO R2 train\t", R2_train)
    println("MIO R2 validation\t", R2_val)
    println("MIO R2 test\t", R2_test)
end;

In [21]:
out_of_sample(X_train1, y_train1, X_val1, y_val1, X_test1, y_test1, β_opt_1, best_K_1, best_Γ1, 0.9)

***PARAMETERS***
Sparsity parameter K = 3
Pairwise correlation threshold ρ = 0.9
Robustness parameter Γ = 0.001

***RESULTS***
MIO R2 train	0.8432352378395275
MIO R2 validation	0.8127748811895958
MIO R2 test	0.7697463987128148


In [22]:
out_of_sample(X_train2, y_train2, X_val2, y_val2, X_test2, y_test2, β_opt_2, best_K_2, best_Γ2, 0.9)

***PARAMETERS***
Sparsity parameter K = 7
Pairwise correlation threshold ρ = 0.9
Robustness parameter Γ = 0.01

***RESULTS***
MIO R2 train	0.9219065010999199
MIO R2 validation	0.8982248481520309
MIO R2 test	0.9274666342602617


In [30]:
a = [16, 18, 19, 22, 24, 27, 28]
for x in a
    println(x%7)
    println(x÷7)
    println()
end

2
2

4
2

5
2

1
3

3
3

6
3

0
4



3